# Exponential Position Sizing for Mean Reversion

In this notebook, we implement a mean reversion strategy that:
1. Scales position size exponentially (instead of linearly) as price moves away from the fair price of 2000
2. Updates positions every X timestamps, where X is a parameter
3. Evaluates performance with different exponential scaling factors

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

## 1. Load Real Data

First, let's load the Squid_Ink price data from the actual data files.

In [ ]:
# Define function to load price data from CSV files
def load_price_data(round_num, day_num):
    import os
    
    # Path to data directory
    data_path = '../../../Prosperity 3 Data'
    
    # Construct file path
    file_path = os.path.join(data_path, f'Round {round_num}/prices_round_{round_num}_day_{day_num}.csv')
    
    # Check if file exists
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return pd.DataFrame()
    
    # Load data
    try:
        data = pd.read_csv(file_path, sep=';')
        print(f"Successfully loaded {len(data)} rows from {file_path}")
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")
        return pd.DataFrame()
    
    return data

# Load data for all days in round 1
print("Loading price data...")
all_data = pd.DataFrame()

for day in range(-2, 1):
    day_data = load_price_data(1, day)
    if len(day_data) > 0:
        # Add day offset to timestamp for continuity
        day_data['timestamp'] += 10**6 * (day+2)
        all_data = pd.concat([all_data, day_data])

# Check if data was loaded successfully
if len(all_data) == 0:
    raise ValueError("No data was loaded. Please check the data directory path.")

In [ ]:
# Check the columns in the loaded data
print(f"Columns in loaded data: {all_data.columns.tolist()}")

# Filter for SQUID_INK
if 'product' in all_data.columns:
    prices = all_data[all_data['product'] == 'SQUID_INK'].copy()
    print(f"Filtered for SQUID_INK using 'product' column: {len(prices)} rows")
elif 'symbol' in all_data.columns:
    prices = all_data[all_data['symbol'] == 'SQUID_INK'].copy()
    print(f"Filtered for SQUID_INK using 'symbol' column: {len(prices)} rows")
else:
    raise ValueError("Could not find 'product' or 'symbol' column in the data.")

# Check if we have any data after filtering
if len(prices) == 0:
    raise ValueError("No SQUID_INK data found after filtering.")

print(f"Loaded {len(prices)} SQUID_INK price data points")

# Sort by timestamp to ensure chronological order
prices = prices.sort_values('timestamp').reset_index(drop=True)
print(f"Sorted data by timestamp")

# Limit to first 20,000 timestamps (in-sample data)
in_sample_prices = prices.iloc[:20000]
print(f"Limited to {len(in_sample_prices)} in-sample data points")

In [ ]:
# Extract price data from the real data
print(f"Available columns in price data: {in_sample_prices.columns.tolist()}")

# Use mid_price as our primary price source
if 'mid_price' in in_sample_prices.columns:
    squid_price = in_sample_prices['mid_price']
    print("Using mid_price column for price data")
# If mid_price is not available, calculate it from bid and ask
elif 'bid_price_1' in in_sample_prices.columns and 'ask_price_1' in in_sample_prices.columns:
    squid_price = (in_sample_prices['bid_price_1'] + in_sample_prices['ask_price_1']) / 2
    print("Calculated mid price from bid_price_1 and ask_price_1")
# Fall back to other price columns if available
elif 'vwap' in in_sample_prices.columns:
    squid_price = in_sample_prices['vwap']
    print("Using vwap column for price data")
elif 'price' in in_sample_prices.columns:
    squid_price = in_sample_prices['price']
    print("Using price column for price data")
else:
    raise ValueError("Could not find appropriate price columns in the data")

print(f"Price range: {squid_price.min()} to {squid_price.max()}")

# Calculate returns
returns = squid_price.pct_change().dropna()
print(f"Calculated {len(returns)} return data points")

# Define the fair price and price range for scaling
FAIR_PRICE = 2000
PRICE_RANGE = 200  # Base price range for scaling

# Create a DataFrame with price and timestamp for easier manipulation
price_df = pd.DataFrame({
    'timestamp': in_sample_prices['timestamp'],
    'price': squid_price,
    'returns': returns
}).dropna().reset_index(drop=True)

### 1.1 Visualize Price Data

Let's visualize the price data and the fair price.

In [ ]:
# Plot price data with fair price
plt.figure(figsize=(12, 6))
plt.plot(price_df.index, price_df['price'], label='Squid_Ink Price')

# Add fair price
plt.axhline(y=FAIR_PRICE, color='r', linestyle='--', label='Fair Price (2000)')
plt.axhline(y=FAIR_PRICE + PRICE_RANGE, color='g', linestyle='--', label='Fair Price + 200')
plt.axhline(y=FAIR_PRICE - PRICE_RANGE, color='g', linestyle='--', label='Fair Price - 200')

plt.title('Squid_Ink Price with Fair Price')
plt.xlabel('Timestamp Index')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()

## 2. Implement Exponential Position Sizing Strategy

Now, let's implement a strategy that scales position size exponentially as price moves away from the fair price.

In [ ]:
def calculate_exponential_position(price, fair_price=2000, price_range=200, exponent=2.0, max_position=1.0):
    """
    Calculate position size that scales exponentially as price moves away from fair price.
    
    Parameters:
        price (float): Current price
        fair_price (float): Fair price (position is 0 at this price)
        price_range (float): Base price range for scaling
        exponent (float): Exponent for scaling (higher = more aggressive exponential growth)
        max_position (float): Maximum position size (absolute value)
        
    Returns:
        float: Position size between -max_position and max_position
    """
    # Calculate normalized deviation from fair price
    normalized_deviation = (price - fair_price) / price_range
    
    # Calculate position size using exponential scaling (negative deviation = positive position)
    if normalized_deviation > 0:
        # Price is above fair price -> short position
        position = -np.sign(normalized_deviation) * min(abs(normalized_deviation) ** exponent, 1.0) * max_position
    else:
        # Price is below fair price -> long position
        position = -np.sign(normalized_deviation) * min(abs(normalized_deviation) ** exponent, 1.0) * max_position
    
    return position

def exponential_position_update_strategy(price_df, update_frequency, exponent=2.0, max_position=1.0, fair_price=2000, price_range=200):
    """
    Implement a strategy that scales position size exponentially and updates positions at specified frequency.
    
    Parameters:
        price_df (pd.DataFrame): DataFrame with price data
        update_frequency (int): Number of timestamps between position updates
        exponent (float): Exponent for scaling (higher = more aggressive exponential growth)
        max_position (float): Maximum position size (absolute value)
        fair_price (float): Fair price (position is 0 at this price)
        price_range (float): Base price range for scaling
        
    Returns:
        pd.Series: Portfolio positions
    """
    # Initialize positions
    positions = pd.Series(0.0, index=price_df.index)
    
    # Calculate initial position
    current_position = calculate_exponential_position(
        price_df['price'].iloc[0], fair_price, price_range, exponent, max_position
    )
    positions.iloc[0] = current_position
    
    # Update positions at specified frequency
    for i in range(1, len(price_df)):
        if i % update_frequency == 0:
            # Update position based on current price
            current_position = calculate_exponential_position(
                price_df['price'].iloc[i], fair_price, price_range, exponent, max_position
            )
        
        positions.iloc[i] = current_position
    
    return positions

### 2.1 Visualize Exponential Position Sizing

Let's visualize how the exponential position sizing function behaves with different exponents.

In [ ]:
# Create a range of prices to visualize position sizing
test_prices = np.linspace(1600, 2400, 1000)  # Prices from 1600 to 2400

# Calculate positions for different exponents
exponents = [0.5, 1.0, 2.0, 3.0]  # Different exponents to test
positions = {}

for exp in exponents:
    positions[exp] = [calculate_exponential_position(price, FAIR_PRICE, PRICE_RANGE, exp) for price in test_prices]

# Plot positions for different exponents
plt.figure(figsize=(12, 6))

for exp in exponents:
    plt.plot(test_prices, positions[exp], label=f'Exponent = {exp}')

# Add reference lines
plt.axhline(y=0, color='k', linestyle='--')
plt.axvline(x=FAIR_PRICE, color='r', linestyle='--', label='Fair Price (2000)')
plt.axvline(x=FAIR_PRICE + PRICE_RANGE, color='g', linestyle='--', label='Fair Price + 200')
plt.axvline(x=FAIR_PRICE - PRICE_RANGE, color='g', linestyle='--', label='Fair Price - 200')

plt.title('Exponential Position Sizing for Different Exponents')
plt.xlabel('Price')
plt.ylabel('Position Size')
plt.ylim(-1.1, 1.1)
plt.legend()
plt.grid(True)
plt.show()

# Compare with linear position sizing
def calculate_linear_position(price, fair_price=2000, price_range=200, max_position=1.0):
    # Calculate normalized deviation from fair price
    normalized_deviation = (price - fair_price) / price_range
    
    # Calculate position size using linear scaling (negative deviation = positive position)
    if normalized_deviation > 0:
        # Price is above fair price -> short position
        position = -min(normalized_deviation, 1.0) * max_position
    else:
        # Price is below fair price -> long position
        position = -max(normalized_deviation, -1.0) * max_position
    
    return position

# Calculate linear positions
linear_positions = [calculate_linear_position(price, FAIR_PRICE, PRICE_RANGE) for price in test_prices]

# Plot comparison of linear vs exponential position sizing
plt.figure(figsize=(12, 6))

plt.plot(test_prices, linear_positions, label='Linear Scaling', linewidth=2)
plt.plot(test_prices, positions[2.0], label='Exponential Scaling (Exponent = 2.0)', linewidth=2)

# Add reference lines
plt.axhline(y=0, color='k', linestyle='--')
plt.axvline(x=FAIR_PRICE, color='r', linestyle='--', label='Fair Price (2000)')
plt.axvline(x=FAIR_PRICE + PRICE_RANGE, color='g', linestyle='--', label='Fair Price + 200')
plt.axvline(x=FAIR_PRICE - PRICE_RANGE, color='g', linestyle='--', label='Fair Price - 200')

plt.title('Linear vs Exponential Position Sizing')
plt.xlabel('Price')
plt.ylabel('Position Size')
plt.ylim(-1.1, 1.1)
plt.legend()
plt.grid(True)
plt.show()

### 2.2 Test Different Parameters

Let's test the strategy with different exponents and update frequencies.

In [ ]:
# Test different parameters
update_frequencies = [1, 10, 50]  # Update frequencies to test
exponents = [0.5, 1.0, 2.0, 3.0]  # Exponents to test

# Initialize results dictionary
results = []

# Store positions for visualization
all_positions = {}

# Test different parameter combinations
for update_freq in update_frequencies:
    for exp in exponents:
        # Run the strategy
        positions = exponential_position_update_strategy(
            price_df, update_freq, exp, max_position=1.0, fair_price=FAIR_PRICE, price_range=PRICE_RANGE
        )
        all_positions[(update_freq, exp)] = positions
        
        # Calculate strategy returns
        strategy_returns = positions.shift(1) * price_df['returns']
        strategy_returns = strategy_returns.dropna()
        
        # Calculate cumulative returns
        cumulative_returns = (1 + strategy_returns).cumprod() - 1
        
        # Calculate performance metrics
        total_return = cumulative_returns.iloc[-1]
        annualized_return = (1 + total_return) ** (252 / len(strategy_returns)) - 1
        annualized_volatility = strategy_returns.std() * np.sqrt(252)
        sharpe_ratio = annualized_return / annualized_volatility if annualized_volatility != 0 else 0
        max_drawdown = (cumulative_returns - cumulative_returns.cummax()).min()
        win_rate = (strategy_returns > 0).mean()
        
        # Calculate position changes for transaction cost analysis
        position_changes = positions.diff().fillna(0)
        num_trades = (position_changes != 0).sum()
        avg_position_size = positions.abs().mean()
        
        # Store results
        results.append({
            'update_frequency': update_freq,
            'exponent': exp,
            'total_return': total_return,
            'annualized_return': annualized_return,
            'annualized_volatility': annualized_volatility,
            'sharpe_ratio': sharpe_ratio,
            'max_drawdown': max_drawdown,
            'win_rate': win_rate,
            'num_trades': num_trades,
            'avg_position_size': avg_position_size
        })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Sort by total return
results_df = results_df.sort_values('total_return', ascending=False)

# Display top 10 results
print("Top 10 Parameter Combinations by Total Return:")
display(results_df.head(10))

### 2.3 Visualize Positions for Different Parameters

Let's visualize how positions change with different exponents and update frequencies.

In [ ]:
# Select a subset of parameters to visualize
best_params = results_df.iloc[0]  # Best parameter combination
best_update_freq = int(best_params['update_frequency'])
best_exponent = best_params['exponent']

print(f"Best Parameters: Update Frequency = {best_update_freq}, Exponent = {best_exponent}")
print(f"Total Return: {best_params['total_return']:.2%}")
print(f"Sharpe Ratio: {best_params['sharpe_ratio']:.2f}")

# Create a figure with two subplots
fig, axes = plt.subplots(2, 1, figsize=(15, 12), sharex=True)

# Plot price
axes[0].plot(price_df.index, price_df['price'], label='Squid_Ink Price')
axes[0].axhline(y=FAIR_PRICE, color='r', linestyle='--', label='Fair Price (2000)')
axes[0].axhline(y=FAIR_PRICE + PRICE_RANGE, color='g', linestyle='--', label='Fair Price + 200')
axes[0].axhline(y=FAIR_PRICE - PRICE_RANGE, color='g', linestyle='--', label='Fair Price - 200')
axes[0].set_title('Squid_Ink Price with Fair Price')
axes[0].set_ylabel('Price')
axes[0].legend()
axes[0].grid(True)

# Plot positions for best parameters
axes[1].plot(price_df.index, all_positions[(best_update_freq, best_exponent)], label=f'Best Parameters (Update Freq = {best_update_freq}, Exponent = {best_exponent})')

# Plot positions for comparison
# Compare with linear scaling (exponent = 1.0)
axes[1].plot(price_df.index, all_positions[(best_update_freq, 1.0)], label=f'Linear Scaling (Update Freq = {best_update_freq}, Exponent = 1.0)', alpha=0.7)

# Compare with different update frequency
different_freq = 1 if best_update_freq > 1 else 10
axes[1].plot(price_df.index, all_positions[(different_freq, best_exponent)], label=f'Different Update Freq (Update Freq = {different_freq}, Exponent = {best_exponent})', alpha=0.7)

axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_title('Positions for Different Parameters')
axes[1].set_xlabel('Timestamp Index')
axes[1].set_ylabel('Position Size')
axes[1].set_ylim(-1.1, 1.1)
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

### 2.4 Visualize Cumulative Returns

Let's visualize the cumulative returns for different parameters.

In [ ]:
# Calculate cumulative returns for selected parameters
cumulative_returns_dict = {}

# Parameters to visualize
params_to_visualize = [
    (best_update_freq, best_exponent),  # Best parameters
    (best_update_freq, 1.0),  # Linear scaling with best update frequency
    (different_freq, best_exponent)  # Best exponent with different update frequency
]

for update_freq, exp in params_to_visualize:
    # Get positions
    positions = all_positions[(update_freq, exp)]
    
    # Calculate strategy returns
    strategy_returns = positions.shift(1) * price_df['returns']
    strategy_returns = strategy_returns.dropna()
    
    # Calculate cumulative returns
    cumulative_returns = (1 + strategy_returns).cumprod() - 1
    cumulative_returns_dict[(update_freq, exp)] = cumulative_returns

# Calculate buy and hold returns for comparison
buy_hold_returns = price_df['returns']
buy_hold_cumulative_returns = (1 + buy_hold_returns).cumprod() - 1

# Plot cumulative returns
plt.figure(figsize=(12, 6))

# Plot buy and hold returns
plt.plot(buy_hold_cumulative_returns, 'k--', label='Buy & Hold')

# Plot returns for selected parameters
for update_freq, exp in params_to_visualize:
    label = f'Update Freq = {update_freq}, Exponent = {exp}'
    if (update_freq, exp) == (best_update_freq, best_exponent):
        label += ' (Best)'
    plt.plot(cumulative_returns_dict[(update_freq, exp)], label=label)

plt.title('Cumulative Returns for Different Parameters')
plt.xlabel('Timestamp Index')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.show()

## 3. Implement Transaction Costs

Now, let's implement transaction costs to make our analysis more realistic.

In [ ]:
# Define transaction cost (1.5/2000 = 0.075% per dollar traded)
transaction_cost = 1.5/2000  # 0.075% per dollar traded

# Initialize results dictionary with transaction costs
results_with_costs = []

for update_freq in update_frequencies:
    for exp in exponents:
        # Get positions
        positions = all_positions[(update_freq, exp)]
        
        # Calculate position changes
        position_changes = positions.diff().fillna(0)
        
        # Calculate transaction costs
        transaction_costs = position_changes.abs() * transaction_cost
        
        # Calculate strategy returns with transaction costs
        strategy_returns_with_costs = positions.shift(1) * price_df['returns'] - transaction_costs.shift(1)
        strategy_returns_with_costs = strategy_returns_with_costs.dropna()
        
        # Calculate cumulative returns with transaction costs
        cumulative_returns_with_costs = (1 + strategy_returns_with_costs).cumprod() - 1
        
        # Calculate performance metrics with transaction costs
        total_return_with_costs = cumulative_returns_with_costs.iloc[-1]
        annualized_return_with_costs = (1 + total_return_with_costs) ** (252 / len(strategy_returns_with_costs)) - 1
        annualized_volatility_with_costs = strategy_returns_with_costs.std() * np.sqrt(252)
        sharpe_ratio_with_costs = annualized_return_with_costs / annualized_volatility_with_costs if annualized_volatility_with_costs != 0 else 0
        max_drawdown_with_costs = (cumulative_returns_with_costs - cumulative_returns_with_costs.cummax()).min()
        win_rate_with_costs = (strategy_returns_with_costs > 0).mean()
        
        # Calculate number of trades and total transaction costs
        num_trades = (position_changes != 0).sum()
        total_transaction_costs = transaction_costs.sum()
        
        # Store results
        results_with_costs.append({
            'update_frequency': update_freq,
            'exponent': exp,
            'total_return': total_return_with_costs,
            'annualized_return': annualized_return_with_costs,
            'annualized_volatility': annualized_volatility_with_costs,
            'sharpe_ratio': sharpe_ratio_with_costs,
            'max_drawdown': max_drawdown_with_costs,
            'win_rate': win_rate_with_costs,
            'num_trades': num_trades,
            'total_transaction_costs': total_transaction_costs
        })

# Convert results to DataFrame
results_with_costs_df = pd.DataFrame(results_with_costs)

# Sort by total return
results_with_costs_df = results_with_costs_df.sort_values('total_return', ascending=False)

# Display top 10 results
print("Top 10 Parameter Combinations by Total Return (with Transaction Costs):")
display(results_with_costs_df.head(10))

### 3.1 Visualize Cumulative Returns with Transaction Costs

Let's visualize the cumulative returns with transaction costs for the best parameters.

In [ ]:
# Get best parameters with transaction costs
best_params_with_costs = results_with_costs_df.iloc[0]
best_update_freq_with_costs = int(best_params_with_costs['update_frequency'])
best_exponent_with_costs = best_params_with_costs['exponent']

print(f"Best Parameters with Transaction Costs: Update Frequency = {best_update_freq_with_costs}, Exponent = {best_exponent_with_costs}")
print(f"Total Return: {best_params_with_costs['total_return']:.2%}")
print(f"Sharpe Ratio: {best_params_with_costs['sharpe_ratio']:.2f}")
print(f"Number of Trades: {best_params_with_costs['num_trades']}")
print(f"Total Transaction Costs: {best_params_with_costs['total_transaction_costs']:.2%}")

# Calculate cumulative returns with transaction costs for selected parameters
cumulative_returns_with_costs_dict = {}

# Parameters to visualize
params_to_visualize = [
    (best_update_freq_with_costs, best_exponent_with_costs),  # Best parameters with costs
    (best_update_freq, best_exponent),  # Best parameters without costs
    (best_update_freq_with_costs, 1.0)  # Linear scaling with best update frequency with costs
]

for update_freq, exp in params_to_visualize:
    # Get positions
    positions = all_positions[(update_freq, exp)]
    
    # Calculate position changes
    position_changes = positions.diff().fillna(0)
    
    # Calculate transaction costs
    transaction_costs = position_changes.abs() * transaction_cost
    
    # Calculate strategy returns with transaction costs
    strategy_returns_with_costs = positions.shift(1) * price_df['returns'] - transaction_costs.shift(1)
    strategy_returns_with_costs = strategy_returns_with_costs.dropna()
    
    # Calculate cumulative returns with transaction costs
    cumulative_returns_with_costs = (1 + strategy_returns_with_costs).cumprod() - 1
    cumulative_returns_with_costs_dict[(update_freq, exp)] = cumulative_returns_with_costs

# Plot cumulative returns with transaction costs
plt.figure(figsize=(12, 6))

# Plot buy and hold returns
plt.plot(buy_hold_cumulative_returns, 'k--', label='Buy & Hold')

# Plot returns for selected parameters
for update_freq, exp in params_to_visualize:
    label = f'Update Freq = {update_freq}, Exponent = {exp}'
    if (update_freq, exp) == (best_update_freq_with_costs, best_exponent_with_costs):
        label += ' (Best with Costs)'
    elif (update_freq, exp) == (best_update_freq, best_exponent):
        label += ' (Best without Costs)'
    plt.plot(cumulative_returns_with_costs_dict[(update_freq, exp)], label=label)

plt.title('Cumulative Returns with Transaction Costs')
plt.xlabel('Timestamp Index')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.show()

### 3.2 Compare Transaction Costs Impact

Let's compare the impact of transaction costs on different parameter combinations.

In [ ]:
# Create a comparison DataFrame
comparison_df = pd.DataFrame()

for update_freq in update_frequencies:
    for exp in exponents:
        # Get results without transaction costs
        without_costs = results_df[(results_df['update_frequency'] == update_freq) & (results_df['exponent'] == exp)].iloc[0]
        
        # Get results with transaction costs
        with_costs = results_with_costs_df[(results_with_costs_df['update_frequency'] == update_freq) & (results_with_costs_df['exponent'] == exp)].iloc[0]
        
        # Calculate impact of transaction costs
        impact = {
            'update_frequency': update_freq,
            'exponent': exp,
            'return_without_costs': without_costs['total_return'],
            'return_with_costs': with_costs['total_return'],
            'return_impact': with_costs['total_return'] - without_costs['total_return'],
            'return_impact_pct': (with_costs['total_return'] - without_costs['total_return']) / abs(without_costs['total_return']) * 100 if without_costs['total_return'] != 0 else 0,
            'sharpe_without_costs': without_costs['sharpe_ratio'],
            'sharpe_with_costs': with_costs['sharpe_ratio'],
            'sharpe_impact': with_costs['sharpe_ratio'] - without_costs['sharpe_ratio'],
            'num_trades': with_costs['num_trades'],
            'total_transaction_costs': with_costs['total_transaction_costs']
        }
        
        # Add to comparison DataFrame
        comparison_df = pd.concat([comparison_df, pd.DataFrame([impact])], ignore_index=True)

# Sort by update frequency and exponent
comparison_df = comparison_df.sort_values(['update_frequency', 'exponent'])

# Display comparison
print("Transaction Costs Impact by Parameter Combination:")
display(comparison_df)

# Create a pivot table to better visualize the impact
pivot_return_impact = comparison_df.pivot(index='update_frequency', columns='exponent', values='return_impact')
pivot_num_trades = comparison_df.pivot(index='update_frequency', columns='exponent', values='num_trades')

# Display pivot tables
print("
Return Impact by Parameter Combination:")
display(pivot_return_impact)

print("
Number of Trades by Parameter Combination:")
display(pivot_num_trades)

## 4. Conclusion

In this notebook, we implemented a mean reversion strategy with exponential position sizing that:
1. Scales position size exponentially (instead of linearly) as price moves away from the fair price of 2000
2. Updates positions every X timestamps, where X is a parameter

Key findings:

1. Exponential position sizing can potentially outperform linear position sizing by taking larger positions when the mispricing is more significant.

2. The optimal exponent depends on the characteristics of the price series. A higher exponent is more aggressive, taking larger positions for smaller deviations from fair price.

3. The update frequency has a significant impact on performance, especially when transaction costs are considered. Less frequent updates generally result in fewer trades and lower transaction costs.

4. Transaction costs have a significant impact on performance, especially for strategies with frequent updates and higher exponents, which tend to generate more trades.

5. The optimal parameter combination with transaction costs tends to have a lower update frequency compared to the optimal combination without transaction costs.

Future improvements could include:

1. Testing different exponential functions or other non-linear scaling functions
2. Implementing adaptive update frequencies based on market conditions
3. Combining with other indicators to filter trades
4. Optimizing the price range parameter
5. Implementing position size limits based on risk management considerations